### First Follow

In [200]:
import os
import pandas as pd
import numpy as np

In [201]:
def checar_dictionario(key, dict):
    check = key in dict
    if check:
        if len(dict[key])>0:
            return True
    return False

In [202]:
def print_dict(dictionary):
    for key, value in dictionary.items():
        print(key + " : " + listToString(value))

In [203]:
file_path = "lib/input/"

try:
    file_name = file_name_externo
except:
    file_name = "exemplo1.txt"
try:
    codigo_entrada = open(file_path + file_name, "r", encoding="utf8").read()
except:
    codigo_entrada = """S -> aSbc | D
D -> dD | d"""

In [204]:
simbolo_inicial = codigo_entrada[0]

In [205]:
codigo_entrada = codigo_entrada.replace('id','i')
codigo_entrada = codigo_entrada.replace('num','n')
codigo_entrada = codigo_entrada.replace(':=','=')

In [206]:
import re

In [207]:
qtd_linhas = codigo_entrada.count("\n") + 1
qtd_linhas

5

In [208]:
### Encontrar
regex_terminais = r"[\[\]a-z+:=*-/;]"
matches = re.finditer(regex_terminais, codigo_entrada.replace("->"," "), re.MULTILINE)
variaveis_terminais = []
for match in matches:
    variaveis_terminais.append(match.group())

variaveis_terminais = list(dict.fromkeys(variaveis_terminais))


In [209]:
### Adicionar num
cond_num_1 = 'n' in variaveis_terminais
cond_num_2 = 'u' in variaveis_terminais
cond_num_3 = 'm' in variaveis_terminais

if (cond_num_1 and cond_num_2 and cond_num_3):
    variaveis_terminais.remove('n')
    variaveis_terminais.remove('u')
    variaveis_terminais.remove('m')
    variaveis_terminais.append('n')

### Adicionar id
cond_id_1 = 'i' in variaveis_terminais
cond_id_2 = 'd' in variaveis_terminais

if (cond_id_1 and cond_id_2):
    variaveis_terminais.remove('i')
    variaveis_terminais.remove('d')
    variaveis_terminais.append('i')

### Adicionar :=
cond_ponto_1 = ':' in variaveis_terminais
cond_ponto_2 = '=' in variaveis_terminais

if (cond_ponto_1 and cond_ponto_2):
    variaveis_terminais.remove(':')
    variaveis_terminais.remove('=')
    variaveis_terminais.append('=')

In [210]:
variaveis_terminais

['b', 'a', 'd', 'g', 'h']

In [211]:
variaveis_terminais

['b', 'a', 'd', 'g', 'h']

In [212]:
# regex_nao_terminais = r"[A-Z]"
regex_nao_terminais = r".+?(?=->)"
matches = re.finditer(regex_nao_terminais, codigo_entrada, re.MULTILINE)
variaveis_nao_terminais = []
for match in matches:
    variaveis_nao_terminais.append(match.group().strip())

variaveis_nao_terminais
# variaveis_nao_terminais = list(dict.fromkeys(variaveis_nao_terminais))

['S', 'A', 'B', 'C']

In [213]:
variaveis_nao_terminais

['S', 'A', 'B', 'C']

In [214]:
variaveis_nao_terminais

['S', 'A', 'B', 'C']

In [215]:
production = dict()

In [216]:

for line in codigo_entrada.splitlines():
    left = line.split('->')[0].strip() 
    right = line.split('->')[1].strip()

    # print(f"FIRST({left}) = ", end="")
    right
    production[left] = []

    for idx, right_part  in enumerate(right.split("|")): 
 
        production[left].append(right_part.strip())
    #     if idx > 0:
    #         print(f" + FIRST({right_part.strip()})", end="")
    #     else:
    #         print(f"FIRST({right_part.strip()})", end="")        

    # print("")



In [217]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [218]:
## Criar dicionário de first
first_dict = production.copy()
follow_dict = production.copy()
follow_position = production.copy()
for elem in first_dict:
    first_dict[elem] = []
    follow_dict[elem] = []
    follow_position[elem] = []

In [219]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [220]:
first_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [221]:
follow_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [222]:
def merge_lists(elem1, elem2):
    if type(elem1 != list):
        elem1 = list(elem1)
    if type(elem2 != list):
        elem2 = list(elem2)
    return list(set(elem1 + elem2))

In [223]:
lista_first_por_elem = []

def first_set(key):
    ### Checar se já existe no dicionário 
    
    resultado = []

    if len(key) == 1 and key in variaveis_nao_terminais:
        if (len(first_dict[key]) != 0):
            return first_dict[key]
            
        value = production[key]
    else:
        value = [key]  

    for elem in value:
        saida = []
        if (len(elem) == 1 and elem == key):
            saida = []
        if elem == 'λ':
            saida = 'λ'
        if elem[0] in variaveis_terminais:
            saida = elem[0]
        if elem[0] in variaveis_nao_terminais:
            if (len(first_dict[elem[0]]) != 0 ):
                check = first_dict[elem[0]]
            else:
                check = first_set(elem[0])
            ### CHecar se o first já foi calculado FIRST T
            if 'λ' not in check:
                saida = first_dict[elem[0]]
            else:       
                if(len(elem) > 1):
                    lista_sem_vazio =  [value for value in first_dict[elem[0]] if value != 'λ']
                    
                    if (elem[1:] == variavel_da_vez):
                        saida = lista_sem_vazio
                    else:
                        # saida = lista_sem_vazio.append(first_set(elem[1:]))
                        saida = merge_lists(lista_sem_vazio, first_set(elem[1:]))
                else:
                    saida = first_dict[elem[0]]
        
        # resultado.append(saida)
        resultado = merge_lists(resultado,saida)
        lista_first_por_elem.append([key,elem,saida])
      
    first_dict[key] = resultado
    return resultado


In [224]:
for variavel in production:
    variavel_da_vez = variavel
    try:
        first_set(variavel)
    except Exception as e:
        print("Erro First: ", e)
        print("Variável: ", variavel_da_vez)


In [225]:
production

{'S': ['ACB', 'CbB', 'Ba'],
 'A': ['da', 'BC'],
 'B': ['g', 'λ'],
 'C': ['h', 'λ']}

In [226]:
follow_dict

{'S': [], 'A': [], 'B': [], 'C': []}

In [227]:
lista_follow_por_elem = []

def follow_set(key):
  resultado = []
  ### Achar elemento que contem key
  for chave, valor in production.items():
    for elem in valor:
      saida = []
      if key == simbolo_inicial:
          saida = "$"
      if key in elem:
        ### Achou elemento que possui a key     
        ### Se elemento estiver na posição final    
        indice = elem.index(key)

        palavra_restante = elem[indice + 1:]

        if palavra_restante == '':
            
          if checar_dictionario(chave,follow_dict):
                saida = follow_dict[chave]
          else:
            if key == chave:
                saida = [] 
            else:
                saida = follow_set(chave)
        else:
            if checar_dictionario(palavra_restante,first_dict):
                saida = first_dict[palavra_restante]
            else:
                saida = first_set(palavra_restante)


      resultado = merge_lists(resultado,saida)     
      if 'λ' in resultado: 
          resultado.remove('λ')
          resultado = merge_lists(resultado,follow_set(chave))
           

  follow_dict[key] = resultado
  lista_follow_por_elem.append([key,elem,saida])
  
  return resultado

In [228]:
for variavel in production:
    variavel_da_vez = variavel
    try:
        follow_set(variavel)
    except Exception as e:
        print("Erro Follow: ", e)
        print("Variável: ", variavel_da_vez)

In [229]:
### Remover chaves de tamanho maior que um

lista_chaves_desnecessarias = [key for key in first_dict if len(key) > 1 or key in variaveis_terminais]
[first_dict.pop(key) for key in lista_chaves_desnecessarias]


[['λ', 'g', 'h'], ['b'], ['a']]

In [230]:
df = pd.DataFrame(columns=['Variavel', 'Producao', 'First', 'Follow'])

In [231]:
df["Variavel"] = variaveis_nao_terminais

In [232]:
def listToString(list):
    str = '{'
    for elem in list:
        str = str + elem + ','

    str = str[:-1] + '}'
    return str


In [233]:
for key, value in production.items():
    df.loc[df['Variavel'] == key, 'Producao'] = listToString(value)

In [234]:
for key, value in first_dict.items():
    df.loc[df['Variavel'] == key, 'First'] = listToString(value)

In [235]:
for key, value in follow_dict.items():
    df.loc[df['Variavel'] == key, 'Follow'] = listToString(value)

In [236]:
nome_arquivo_saida = f'lib/output/{file_name[:-3]}csv'
try:
    df.to_csv(nome_arquivo_saida, index = False)
except:
    print("Não pode salvar aquivo de saída")

In [237]:
df

,Variavel,Producao,First,Follow
0,S,"{ACB,CbB,Ba}","{a,λ,b,g,d,h}",{$}
1,A,"{da,BC}","{λ,g,d,h}","{g,$,h}"
2,B,"{g,λ}","{λ,g}","{g,a,$,h}"
3,C,"{h,λ}","{λ,h}","{h,g,$,b}"


#### Primeira Tabela Resultante

In [238]:
colunas = variaveis_terminais.copy()
colunas.insert(0,'M(V,T)')

In [239]:
df_1 = pd.DataFrame(columns=colunas)
df_1['M(V,T)'] = variaveis_nao_terminais
df_1.index = df_1['M(V,T)']

In [240]:
df_1

,"M(V,T)",b,a,d,g,h
"M(V,T)",,,,,,
S,S,NaN,NaN,NaN,NaN,NaN
A,A,NaN,NaN,NaN,NaN,NaN
B,B,NaN,NaN,NaN,NaN,NaN
C,C,NaN,NaN,NaN,NaN,NaN


In [241]:
print_dict(production)

S : {ACB,CbB,Ba}
A : {da,BC}
B : {g,λ}
C : {h,λ}


In [242]:
### Todos as produções vão para seu FIRST
### As Vazio vão para o follow

In [243]:
### Tratar casos onde a produção é vazia -> Usar elementos do follow

for row in lista_first_por_elem:
    if 'λ' in row[1]:
        lista_follow = follow_set(row[0])
        lista_follow = [elem for elem in lista_follow if elem not in first_dict[row[0]]]
        lista = merge_lists(row[2],lista_follow)        
        row[2] = [elem for elem in lista if elem != 'λ']
    else:
        if 'λ' in row[2]:
            lista_follow = follow_set(row[0])
            lista_follow = [elem for elem in lista_follow if elem not in first_dict[row[0]]]
            lista = merge_lists(row[2],lista_follow)        
            row[2] = [elem for elem in lista if elem != 'λ']
        

### Eliminar Variaves Não Terminais Erradas

lista_table = [elem for elem in lista_first_por_elem if elem[0] in variaveis_nao_terminais]

In [244]:
for elem in lista_table:
    for coluna in elem[2]:
        df_1.loc[elem[0], coluna] = elem[0] + "->" + elem[1]
    

In [245]:
### Formatar

df_1 = df_1.fillna('-')
df_1

,"M(V,T)",b,a,d,g,h,$
"M(V,T)",,,,,,,
S,S,S->CbB,S->Ba,S->ACB,S->Ba,S->CbB,S->ACB
A,A,-,-,A->da,A->BC,A->BC,A->BC
B,B,-,B->λ,-,B->g,B->λ,B->λ
C,C,C->λ,-,-,C->λ,C->h,C->λ


In [246]:
#### Tabela de Verificação
palavra = 'dahg'

In [247]:
def juntar_lista(lista):
    saida = ''
    for elem in lista:
        saida = saida + elem
    return saida

In [278]:
df_2 = pd.DataFrame(columns=['Pilha','Entrada','Producao'])

lista_pilha= ['$']
lista_entrada = ['$']

df_2.loc[0] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),simbolo_inicial]



### Colocar Produção da linha passada na pilha
producao = simbolo_inicial

i = 1

nova_pilha = producao.split('->')[-1]
nova_entrada = palavra

[lista_pilha.insert(len(lista_pilha), elem) for elem in nova_pilha[::-1]]
[lista_entrada.insert(len(lista_entrada) - 1, elem) for elem in nova_entrada]

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]

if (elemento_analisado == elemento_entrada):
    producao = ''
    # lista_pilha.pop()
else:
    producao = df_1.loc[elemento_analisado][elemento_entrada]
    if (len(producao) > 1):
        print(producao)
        # lista_pilha.pop()
    else:
        print('break')

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]

lista_pilha.pop()

i = 2

nova_pilha = producao.split('->')[-1]
nova_entrada = palavra


[lista_pilha.insert(len(lista_pilha), elem) for elem in nova_pilha[::-1]]
# [lista_entrada.insert(len(lista_entrada) - 1, elem) for elem in nova_entrada]

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]



if (elemento_analisado == elemento_entrada):
    producao = ''
    # lista_pilha.pop()
else:
    producao = df_1.loc[elemento_analisado][elemento_entrada]
    if (len(producao) > 1):
        print(producao)
        # lista_pilha.pop()
    else:
        print('break')

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]
lista_pilha.pop()

i = 3

nova_pilha = producao.split('->')[-1]

[lista_pilha.insert(len(lista_pilha), elem) for elem in nova_pilha[::-1]]

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]



if (elemento_analisado == elemento_entrada):
    producao = ''
    # lista_pilha.pop()
else:
    producao = df_1.loc[elemento_analisado][elemento_entrada]
    if (len(producao) > 1):
        print(producao)
        # lista_pilha.pop()
    else:
        print('break')

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]
eliminado = lista_pilha.pop()

i = 4

nova_pilha = producao.split('->')[-1]

[lista_pilha.insert(len(lista_pilha), elem) for elem in nova_pilha[::-1]]

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]

if (eliminado == elemento_entrada):
    producao = ''
    lista_entrada.pop(0)

    print('Here')
else:
    producao = df_1.loc[eliminado][elemento_entrada]
    if (len(producao) > 1):
        print(producao)
    else:
        print('break')

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]
eliminado = lista_pilha.pop()

print(f'''
    lista_pilha         = {lista_pilha}
    lista_entrada       = {lista_entrada}
    elemento_analisado  = {elemento_analisado}
    elemento_entrada    = {elemento_entrada}
    producao            = {producao}
    nova_pilha          = {nova_pilha}
    nova_entrada        = {nova_entrada}
''')

i = 5

nova_pilha = producao.split('->')[-1]

[lista_pilha.insert(len(lista_pilha), elem) for elem in nova_pilha[::-1]]

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]

if (eliminado == elemento_entrada):
    producao = ''
    lista_entrada.pop(0)

    print('Here')
else:
    producao = df_1.loc[eliminado][elemento_entrada]
    if (len(producao) > 1):
        print(producao)
    else:
        print('break')

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]
eliminado = lista_pilha.pop()
df_2



S->ACB
A->da
Here

    lista_pilha         = ['$', 'B', 'C']
    lista_entrada       = ['a', 'h', 'g', '$']
    elemento_analisado  = a
    elemento_entrada    = d
    producao            = 
    nova_pilha          = 
    nova_entrada        = dahg

Here


,Pilha,Entrada,Producao
0,$,$,S
1,$S,dahg$,S->ACB
2,$BCA,dahg$,A->da
3,$BCad,dahg$,
4,$BCa,ahg$,
5,$BC,hg$,


In [256]:
lista_entrada       = ['d', 'a', 'h', 'g', '$']
lista_entrada.pop()

'$'

In [183]:
lista_pilha

['S', 'S', '$']

In [182]:
[lista_pilha.insert(0, elem) for elem in simbolo_inicial]

[None]

In [171]:
lista_pilha

['$', '', 'BC', 'ad']

In [179]:
lista_entrada = ['$']
entrada = 'dahg'

[lista_entrada.insert(len(lista_entrada) - 1, elem) for elem in entrada]
lista_entrada

['d', 'a', 'h', 'g', '$']

In [166]:
elemento_analisado

'd'

In [167]:
elemento_entrada

'd'

In [155]:
merge_lists(['$'],['BCA'])

['BCA', '$']

In [161]:
lista_pilha

['$', 'BC']

In [152]:
lista_pilha

['$', 'BCA']

In [141]:
lista_pilha[-1][-1]

'A'

In [137]:
elemento_analisado

'BCA'

In [135]:
lista_entrada[0]

'ahg'

In [134]:
lista_entrada[0] = lista_entrada[0][1:]

In [129]:
lista_pilha

['$', 'BCA']

In [118]:
elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0][0]

condicao = len(df_1.loc[elemento_analisado][elemento_entrada]) > 1

In [108]:
df_1[lista_pilha[-1]]

KeyError: 'S'

In [61]:
df_1[df_1['M(V,T)'] == lista_pilha[-1]]

,"M(V,T)",b,a,d,g,h,$
0,S,S->CbB,S->Ba,S->ACB,S->Ba,S->CbB,S->ACB


In [1017]:
lista_entrada[0][0]

'd'

In [1018]:
cond = df_1['M(V,T)'] == simbolo_inicial
producao = df_1[cond][lista_entrada[0][0]]
producao

0    S->ACB
Name: d, dtype: object

In [999]:
df_2 = pd.DataFrame(columns=['Pilha','Entrada','Producao'])

lista_pilha = ['$']
lista_entrada = ['$']

pilha = [p for p in lista_pilha]
df_2.loc[0] = ['$','$',simbolo_inicial]

#passo1

i = 0

while range(0,10):

    lista_pilha

    pilha_anterior = df_2['Pilha'][indice]
    entrada_anterior = df_2['Entrada'][indice]

    if i == 0:
        pilha = pilha_anterior + simbolo_inicial
    else:
        pilha = 
    entrada = palavra + entrada_anterior


cond = df_1['M(V,T)'] == 'S'
producao = df_1[cond][entrada[0]][0]

if producao == '-':
    break
else:


df_2.loc[indice + 1] = [pilha,entrada,producao]

df_2

,Pilha,Entrada,Producao
0,$,$,S
1,$S,dahg$,S->ACB


In [1000]:
df_1

,"M(V,T)",b,a,d,g,h,$
0,S,S->CbB,S->Ba,S->ACB,S->Ba,S->CbB,S->ACB
1,A,-,-,A->da,A->BC,A->BC,A->BC
2,B,-,B->λ,-,B->g,B->λ,B->λ
3,C,C->λ,-,-,C->λ,C->h,C->λ


In [997]:
cond = df_1['M(V,T)'] == 'S'
df_1[cond][entrada[0]][0]

'S->ACB'

In [993]:
cond = df_1['M(V,T)'] == 'S'
df_1[cond]['d']

0    S->ACB
Name: d, dtype: object

In [991]:
df_1

,"M(V,T)",b,a,d,g,h,$
0,S,S->CbB,S->Ba,S->ACB,S->Ba,S->CbB,S->ACB
1,A,-,-,A->da,A->BC,A->BC,A->BC
2,B,-,B->λ,-,B->g,B->λ,B->λ
3,C,C->λ,-,-,C->λ,C->h,C->λ


In [980]:
df_2.index.max()

0

In [ ]:
df_2 = pd.DataFrame(columns=['Pilha','Entrada','Producao'])

lista_pilha= ['$']
lista_entrada = ['$']

df_2.loc[0] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),simbolo_inicial]


i = 1

[lista_pilha.insert(len(lista_pilha), elem) for elem in simbolo_inicial[::-1]]
[lista_entrada.insert(len(lista_entrada) - 1, elem) for elem in palavra]


### Checar se o ultimo elemento da pilha, gera o primeiro da entrada
elemento_analisado = lista_pilha[-1][-1]
elemento_entrada = lista_entrada[0][0]

producao = df_1.loc[elemento_analisado][elemento_entrada]
condicao = len(producao) > 1

i = 2

df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]


string_entrada = producao.split('->')[-1]

lista_pilha.pop()
[lista_pilha.insert(len(lista_pilha), elem) for elem in string_entrada[::-1]]
# lista_pilha[-1] = lista_pilha[-1][:-1]
# lista_pilha.insert(len(lista_pilha),string_entrada[::-1])

elemento_analisado = lista_pilha[-1]
elemento_entrada = lista_entrada[0]

### Checar se pode ser retirado algum elemento da entrada
if (elemento_analisado == elemento_entrada):
    lista_pilha.pop()
    lista_entrada.pop(0)

# # # lista_entrada.insert(0,palavra)
print(f'''
    lista_pilha         = {lista_pilha}
    lista_entrada       = {lista_entrada}
    elemento_analisado  = {elemento_analisado}
    elemento_entrada    = {elemento_entrada}
    producao            = {producao}
    string_entrada      = {string_entrada}
''')
# producao = df_1.loc[elemento_analisado][elemento_entrada]
# condicao = len(producao) > 1

# i = 3

# df_2.loc[i] = [juntar_lista(lista_pilha),juntar_lista(lista_entrada),producao]

# string_entrada = producao.split('->')[-1]
# print(string_entrada)
# lista_pilha[-1] = lista_pilha[-1][:-1]
# lista_pilha.insert(len(lista_pilha),string_entrada[::-1])

# elemento_analisado = lista_pilha[-1][-1]
# elemento_entrada = lista_entrada[0][0]

# ### Checar se pode ser retirado algum elemento da entrada
# if (elemento_analisado == elemento_entrada):
#     lista_pilha[-1] = lista_pilha[-1][:-1]
#     lista_entrada[0] = lista_entrada[0][1:]
# # # lista_entrada.insert(0,palavra)
# producao = df_1.loc[elemento_analisado][elemento_entrada]
# condicao = len(producao) > 1

df_2

